## <span style="color:#ff5f27;">🚀 Batch Inference Pipeline</span>

This notebook performs the following actions:

* Gets a feature view object with its name/version from Hopsworks
* Downloads a Pandas DataFrame with new inference data from Hopsworks using the feature view and the call `fv.get_batch_data(start_time="...")`.
* Downloads the model from Hopsworks using with its name/version.
* Makes predictions on batch data.

## <span style="color:#ff5f27;">📝 Imports </span>


In [ ]:
!pip install --quiet hopsworks 

In [ ]:
import hopsworks
import pandas as pd
import joblib
import os
import numpy as np
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot
import seaborn as sns

In [ ]:
# Define version numbers for feature view and model
fv_version = 1
model_version = 1

# Define start and end times for the data
start_time_data = "2016-11-01"
end_time_data = "2016-12-01"

## <span style="color:#ff5f27;"> 🔮 Connect to Hopsworks Feature Store</span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## <span style="color:#ff5f27;"> ⚙️ Feature View Retrieval</span>


In [ ]:
# Get the 'loans_approvals' feature view
feature_view = fs.get_feature_view(
    name="loans_approvals", 
    version=fv_version,
)

## <span style="color:#ff5f27;">🗄 Model Registry</span>


In [ ]:
# Get the model registry
mr = project.get_model_registry()

## <span style='color:#ff5f27'>🚀 Fetch and test the model</span>

In [ ]:
# Retrieve the model from the Model Registry using the name "lending_model" and specified version
model = mr.get_model(
    "lending_model",
    version=model_version,
)

# Download the model directory from the Model Registry
model_dir = model.download()

# Load the model using joblib from the downloaded model directory
model = joblib.load(model_dir + "/lending_model.pkl")

## <span style="color:#ff5f27;">🔮  Batch Prediction </span>

In [ ]:
# Initialize batch scoring
feature_view.init_batch_scoring(1)

# Get batch data for a specified time range from start_time_data to end_time_data
batch_data = feature_view.get_batch_data(
    start_time=start_time_data,
    end_time=end_time_data,
)

# Display the first three rows of the batch data
batch_data.head(3)

In [ ]:
# Make predictions on the batch data using the loaded model
predictions = model.predict(batch_data)

# Display the first 10 predictions
predictions[:10]

---

### <span style="color:#ff5f27;">🥳 <b> Next Steps  </b> </span>
Congratulations you've now completed the Loan Approval tutorial for Managed Hopsworks.

Check out our other tutorials on ➡ https://github.com/logicalclocks/hopsworks-tutorials

Or documentation at ➡ https://docs.hopsworks.ai